In [230]:
# data year/netcdf test


import numpy as np
import pandas as pd
import datetime as dt
import shutil
import os
import copy
import imp
from invoke import run
from numba import jit
from scipy import stats
from dateutil.relativedelta import relativedelta
from mpl_toolkits.basemap import Basemap

import data_year as dy
import grid_set as gs
from netCDF4 import Dataset

imp.reload(dy)

<module 'data_year' from '/Users/H/WAVES/geo_data_group/data_year.py'>

In [253]:
d_in = np.ones([14,10,10])
d_in[1,:,:] = 2.0
dates = [dt.datetime(1999,12,1) + relativedelta(months=m) for m in range(14)]
test_dy = dy.data_year(d_in,dates,12)

In [254]:
test_dy.build_static_mask(np.ones([10,10],dtype=bool))
test_dy.mask[:,4:6,4:6] = False

In [238]:
test_ma = np.ma.array(test_dy.data)

In [61]:
test_ma.mask = test_dy.mask==False

In [109]:
NC_f = Dataset('nc_test.nc', 'w', format='NETCDF4')

In [110]:
NC_f.createDimension('time', test_dy.n_t)
NC_f.createDimension('x', test_dy.m)
NC_f.createDimension('y', test_dy.n)
NC_f.createDimension('periods', test_dy.periods)
NC_f.createDimension('nyrs', test_dy.nyrs)

<class 'netCDF4._netCDF4.Dimension'>: name = 'nyrs', size = 1

In [111]:
data_name  = 'test_D'
DY_data = NC_f.createVariable(data_name, 'f4', ('time','x','y'))
DY_mask = NC_f.createVariable('mask', 'i1', ('time','x','y'))
DY_time = NC_f.createVariable('time', 'f4', ('time',))
DY_yrpd = NC_f.createVariable('dy_yrpd', 'i8', ('nyrs','periods'))
DY_yrpd_mask = NC_f.createVariable('dy_yrpd_mask', 'i1', ('nyrs','periods'))
        

In [112]:
DY_data[:] = test_dy.data
DY_mask[:] = test_dy.mask
temp_time = [test_dy.dates[t].toordinal() for t in range(test_dy.n_t)]
DY_time[:] = temp_time
DY_yrpd[:] = test_dy.yrpd
DY_yrpd_mask[:] = test_dy.yrpd.mask


In [84]:
DY_data.mask

True

In [113]:
NC_f.close()

In [255]:
test_dy.yrpd

masked_array(
  data=[[--, --, --, --, --, --, --, --, --, --, --, 0],
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        [13, --, --, --, --, --, --, --, --, --, --, --]],
  mask=[[ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False],
        [False, False, False, False, False, False, False, False, False,
         False, False, False],
        [False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True]],
  fill_value=999999)

In [301]:
mask,y0,yE,t0,tE = dy.get_range_mask(test_dy,False,[0,2],[13,13])
np.shape(mask)

(14, 10, 10)

In [257]:
y0,yE,yE-y0

(0, 2, 2)

In [222]:
type(test_dy.mask)==np.ndarray

True

In [292]:
yrpd_cp = np.ma.empty([yE-y0+1,test_dy.periods],dtype = int)
yrpd_cp[:,:] = test_dy.yrpd[y0:yE+1,:]
yrpd_cp.mask[yrpd_cp>tE] = True
yrpd_cp.mask[yrpd_cp<t0] = True
# save the appropritate depending on the y0 etc
# [t for t in yrpd_cp[y0:yE+1,:].compressed()]
yrpd_cp

masked_array(data=[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]],
             mask=[[False, False, False, False, False, False, False,
                    False, False, False, False, False]],
       fill_value=999999)

In [302]:
np.min(yrpd_cp),t0

# given the limits what is the new yrpd
yrpd_cp = np.ma.empty([yE-y0+1,test_dy.periods],dtype = int)
yrpd_cp[:,:] = test_dy.yrpd[y0:yE+1,:]
yrpd_cp.mask[yrpd_cp>tE] = True
yrpd_cp.mask[yrpd_cp<t0] = True
# this finds minimum value
tt0 = np.maximum(np.min(yrpd_cp),t0)
ttE = np.max(yrpd_cp)
# find empty rows
iy = dates[tt0].year - dates[0].year
ey = dates[-1].year - dates[ttE].year
# cut them
yrpd_cp[(iy>y0)*iy:yE+1-ey,:] - tt0
# yE-(yE-iy),yE+1-ey

masked_array(data=[[0, --, --, --, --, --, --, --, --, --, --, --]],
             mask=[[False,  True,  True,  True,  True,  True,  True,
                     True,  True,  True,  True,  True]],
       fill_value=999999)

In [300]:
(iy>y0)*iy

1

In [235]:
test_dy.save_nc('test_nc_auto.nc',time_set=[1,2])